In [ ]:
# The aim of this module is to assess information completeness

# File name convention
# R_ = raw extract 
# E_ = Entry files into enrichment layer
# C_ = enriched layer with calculation
# I = insights layer, designed for model baselines
# V_ = validation

In [1]:
#Develop ticker coverage table
import pandas as pd
import numpy as np

TIC = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/R_ALL_TICKER.pkl') 

TIC_U = TIC.drop_duplicates(subset=['colummn']).rename(columns={'colummn': 'TickName'})[['TickName']]

TIC_U['Counter'] = 1

In [4]:

# INF = pd.read_excel (r'/Users/joezhou/Downloads/R_ALL_INFO.xlsx') 

DF_YFIN_INFO1 = pd.read_csv (r'/Users/joezhou/Downloads/Investment/1 data/R_ALL_INFO_1.csv',sep='|') 
DF_YFIN_INFO2 = pd.read_csv (r'/Users/joezhou/Downloads/Investment/1 data/R_ALL_INFO_2.csv',sep='|') 
DF_YFIN_INFO3 = pd.read_csv (r'/Users/joezhou/Downloads/Investment/1 data/R_ALL_INFO_3.csv',sep='|') 
DF_YFIN_INFO4 = pd.read_csv (r'/Users/joezhou/Downloads/Investment/1 data/R_ALL_INFO_4.csv',sep='|') 

INF = pd.concat([DF_YFIN_INFO1, DF_YFIN_INFO2, DF_YFIN_INFO3, DF_YFIN_INFO4], ignore_index=True)

INF_U = INF.drop_duplicates(subset=['symbol']).rename(columns={'symbol': 'TickName'})[['TickName']]
INF_U['IN_INFO']="In Info Extract"
INF_U['F_IN_INFO']= 1


PRICE = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/R_ALL_Prices.pkl') 
PRICE_U = PRICE.drop_duplicates(subset=['TickName'])[['TickName']]
PRICE_U['IN_PRICE']="In Price Extract"
PRICE_U['F_IN_PRICE']= 1

# FIN = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/R_ALL_FIN.pkl') 
FIN = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/R_ALL_FIN_1.pkl') 
FIN_U = FIN.drop_duplicates(subset=['TickName'])[['TickName']]
FIN_U['IN_FIN']="In Financial Extract"
FIN_U['F_IN_FIN']= 1

DIV = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/R_ALL_DIV.pkl') 
DIV_U = DIV.drop_duplicates(subset=['TickName'])[['TickName']]
DIV_U['IN_DIV']="In Dividend Extract"
DIV_U['F_IN_DIV']= 1


In [5]:
# Join it back together
TIC_COVERAGE = TIC_U.merge(INF_U,how='left',on='TickName').merge(PRICE_U,how='left',on='TickName').merge(FIN_U,how='left',on='TickName').merge(DIV_U,how='left',on='TickName').fillna(0)

TIC_COVERAGE['COUNT_SOURCE'] = TIC_COVERAGE['F_IN_INFO']+TIC_COVERAGE['F_IN_PRICE']+TIC_COVERAGE['F_IN_FIN']+TIC_COVERAGE['F_IN_DIV']

Tbl_Coverage = TIC_COVERAGE.groupby(['IN_INFO','IN_PRICE','IN_FIN','IN_DIV'])['Counter'].count().reset_index().sort_values(by=['Counter'],ascending=False)
Tbl_Coverage['Percentage'] = 100 * Tbl_Coverage['Counter']  / Tbl_Coverage['Counter'].sum()
Tbl_Coverage.head(27)
# To-do: add market cap data in here to quantify the proportion missing

,IN_INFO,IN_PRICE,IN_FIN,IN_DIV,Counter,Percentage
4,In Info Extract,In Price Extract,0,0,998,44.139761
5,In Info Extract,In Price Extract,0,In Dividend Extract,550,24.325520
6,In Info Extract,In Price Extract,In Financial Extract,0,326,14.418399
7,In Info Extract,In Price Extract,In Financial Extract,In Dividend Extract,190,8.403361
0,0,0,0,0,157,6.943830
1,0,In Price Extract,0,0,21,0.928793
3,In Info Extract,0,0,0,13,0.574967
2,0,In Price Extract,0,In Dividend Extract,6,0.265369


In [6]:
# Only keep columns needed, selected after manual review

TIC_COVERAGE_TRIM = TIC_COVERAGE[['TickName', 'COUNT_SOURCE','IN_INFO','IN_PRICE','IN_DIV','IN_FIN']]

TIC_COVERAGE_TRIM.to_pickle("/Users/joezhou/Downloads/Investment/1 data/E_TICK_INPUTS.pkl")
